In [6]:
import pandas as pd
import os
import re
import glob

In [19]:
fpath = "/scratch/indikar_root/indikar1/shared_data/adapter_clean_up/reports/adapter_summary.csv"

df = pd.read_csv(fpath)
print(f"{df.shape=}")
df.head()

df.shape=(37432, 3)


,Name,count,basename
0,SQK-MAP006_Y_Bottom_SK64,821562,o4b01.txt
1,BC01_rev,812206,o4b01.txt
2,BC01,401603,o4b01.txt
3,PCR_2_start,392930,o4b01.txt
4,PCR_1_end,51469,o4b01.txt


In [53]:
def categorize_item(item):
    """
    Categorizes an item as "barcode", "PCR", or "adapter" based on its prefix.

    Args:
        item (str): The item to categorize.

    Returns:
        str: The category of the item.
    """

    if "BC" in item:
        return "Barcode"
    else:
        return "PCR"
    
df['adapter_type'] = df['Name'].apply(lambda x: categorize_item(x))
df['sample'] = df['basename'].str[:2]
df['cell'] = df['basename'].str[2:5]
df.head()

,Name,count,basename,adapter_type,sample,cell
0,SQK-MAP006_Y_Bottom_SK64,821562,o4b01.txt,PCR,o4,b01
1,BC01_rev,812206,o4b01.txt,Barcode,o4,b01
2,BC01,401603,o4b01.txt,Barcode,o4,b01
3,PCR_2_start,392930,o4b01.txt,PCR,o4,b01
4,PCR_1_end,51469,o4b01.txt,PCR,o4,b01


In [54]:
gx = df.groupby('Name')['count'].sum()
gx = gx.sort_values(ascending=False)
gx.head(10)

Name
SQK-MAP006_Y_Bottom_SK64    102678625
PCR_2_start                  88690883
PCR_1_end                    39140260
PCR_1_start                  36673957
BC11_rev                     17282755
BC68_rev                     13526010
BC11                         12076653
BC68                         11059191
BC55_rev                      8368905
BC55                          8320134
Name: count, dtype: int64

In [57]:
gx = df.groupby(['sample', 'adapter_type']).agg(
    total_instances = ('count', 'sum'),
).reset_index()


gx = pd.pivot(gx, index='sample', columns='adapter_type', values='total_instances')
gx['Total Detected'] = gx.sum(axis=1)
gx['Barcode'] = (gx['Barcode'] / gx['Total Detected']) * 100
gx['PCR'] = (gx['PCR'] / gx['Total Detected']) * 100
gx = gx.reset_index()

print(gx.round(2).astype(str).to_latex(index=False))

\begin{tabular}{llll}
\toprule
sample & Barcode & PCR & Total Detected \\
\midrule
o1 & 41.43 & 58.57 & 37733663 \\
o2 & 45.21 & 54.79 & 80944949 \\
o3 & 45.44 & 54.56 & 269214162 \\
o4 & 44.99 & 55.01 & 104288216 \\
\bottomrule
\end{tabular}



In [20]:
df['Name'].unique()

array(['  SQK-MAP006_Y_Bottom_SK64', '  BC01_rev', '  BC01',
       '  PCR_2_start', '  PCR_1_end', '  PCR_1_start', '  BC93_rev',
       '  BC93', '  BC03_rev', '  BC79_rev', '  BC79', '  BC03',
       '  BC55_rev', '  BC55', '  BC24_rev', '  BC09', '  BC68_rev',
       '  BC68', '  BC25_rev', '  BC25', '  BC24', '  BC65', '  BC09_rev',
       '  BC73_rev', '  SQK-MAP006_Short_Y_Top_LI32', '  BC73',
       '  BC06_rev', '  BC65_rev', '  SQK-MAP006_Short_Y_Bottom_LI33',
       '  BC05_rev', '  PCR_3_end', '  BC02', '  BC04', '  BC04_rev',
       '  PCR_3_start', '  BC06', '  BC14', '  BC10', '  BC02_rev',
       '  BC05', '  BC13', '  BC21', '  BC19', '  BC76', '  BC12_rev',
       '  BC45', '  BC15', '  BC23', '  BC10_rev', '  BC08',
       '  SQK-NSK007_Y_Top', '  BC17', '  BC16', '  BC08_rev', '  BC43',
       '  BC46', '  BC12', '  BC13_rev', '  BC20', '  BC34', '  BC11',
       '  BC89', '  BC84', '  BC15_rev', '  BC07', '  BC21_rev', '  BC81',
       '  BC63', '  BC82', '  BC33',